In [1]:
%load_ext autoreload
%autoreload 2

# Typewriter: 26 Tools

Let's see how to evaluate an agent's ability to use tools.

    A task where the agent must type a given string one letter at a time.

    In this variation of the task, the agent is given access to 26 parameterless functions,
    each representing a letter of the alphabet.

In [2]:
from langchain_benchmarks import clone_public_dataset, registry

In [3]:
task = registry["Tool Usage - Typewriter (26 tools)"]
task

Name,Tool Usage - Typewriter (26 tools)
Type,ToolUsageTask
Dataset ID,https://smith.langchain.com/public/128af05e-aa00-4e3b-a958-d166dd450581/d
Description,"Environment with 26 tools each tool represents a letter of the alphabet. The objective of this task is to evaluate the model's ability the use tools for a simple repetition task. For example, if the string is 'abc', the tools 'a', 'b', and 'c' must be invoked in that order. The dataset includes examples of varying difficulty. The difficulty is measured by the length of the string. This is a variation of the typer writer task, where 26 parameterless tools are given instead of a single tool that takes a letter as an argument."


Clone the dataset associaetd with this task

In [4]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Tool Usage - Typewriter (26 tools) already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/5051c0ae-16be-4afa-b914-84acbc5e9659.


Let's build an agent that we can use for evaluation.

In [5]:
env = task.create_environment()

In [8]:
from langchain_benchmarks.tool_usage import agents

agent_factory = agents.OpenAIAgentFactory(task, model="gpt-3.5-turbo-16k")

# Let's test that our agent works
agent = agent_factory()
agent.invoke({"question": "foo"})

{'question': 'foo',
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='_Exception', tool_input='Invalid or incomplete response', log="Could not parse tool input: {'arguments': 'f', 'name': 'f'} because the `arguments` is not valid JSON."),
   'Invalid or incomplete response'),
  (AgentActionMessageLog(tool='f', tool_input={}, log='\nInvoking: `f` with `{}`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '', 'name': 'f'}})]),
   'OK'),
  (AgentActionMessageLog(tool='o', tool_input={}, log='\nInvoking: `o` with `{}`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '', 'name': 'o'}})]),
   'OK'),
  (AgentActionMessageLog(tool='o', tool_input={}, log='\nInvoking: `o` with `{}`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '', 'name': 'o'}})]),
   'OK'),
  (AgentActionMessageLog(tool='f', tool_input=

## Eval

Let's evaluate an agent now

In [ ]:
from langsmith.client import Client

from langchain_benchmarks.tool_usage import STANDARD_AGENT_EVALUATOR

client = Client()

test_run = client.run_on_dataset(
    dataset_name=task.name,
    llm_or_chain_factory=agent_factory.create,
    evaluation=STANDARD_AGENT_EVALUATOR,
    verbose=True,
    tags=["gpt-3.5-turbo-16k"],
)